## Structured Streaming - Exercicio - Aula 9

1. Criar uma aplicação em scala usando o spark para ler os dados da porta 9999 e exibir no console

CRIAR LEITURA STREAM USANDO PYSPARK AO INVES DE SPARK-SHELL(SCALA)
porta_leitura = spark.readStream.format("socket").option("host","localhost").option("port","9999").load()
porta_saida = porta_leitura.writeStream.format("console").start()

UTILIZANDO NETCAT DENTRO DO CONTAINER JUPYTER PARA INJETAR MENSAGENS
nc -lp 9999

2. Ler os arquivos csv “hdfs://namenode:8020/user/<nome>/data/iris/*.data” em modo streaming com o seguinte schema:
    sepal_length float
    sepal_width float
    petal_length float
    petal_width float
    class string
3. Visualizar o schema das informações
4. Salvar os dados no diretório “hdfs://namenode:8020/user/<nome>/stream_iris/path” e o checkpoint em “hdfs://namenode:8020/user/<nome>/stream_iris/check”
5. Verificar a saida no hdfs e entender como os dados foram salvos
6. Bônus: Contar as palavras do exercício 1.

In [ ]:
!hdfs dfs -ls /user/ronnan/data/iris/

In [ ]:
#verificar se os arquivos com final .data estão dentro da pasta
## iris = spark.read.csv("/user/ronnan/data/iris/*.data").show(5)

In [ ]:
#verificar o total de linhas contidas no arquivo dezkiris
#iris = spark.read.csv("/user/ronnan/data/iris/bezdekIris.data")
#iris.count()

In [ ]:
#verificar o total de linhas contidas no arquivo dezkiris
#iris = spark.read.csv("/user/ronnan/data/iris/iris.data")
#iris.count()

In [ ]:
#necessario importa bibliotecas sql.types parar criar schema
from pyspark.sql.types import StructType

In [ ]:
#necessario criar schema antes de ler os arquivos como readStream
iris_schema = StructType()\
    .add("sepal_length", "float")\
    .add("sepal_width", "float")\
    .add("petal_length", "float")\
    .add("petal_width", "float")\
    .add("class", "string")

In [ ]:
#verificar se o schema foi criado corretamente
print(iris_schema)

In [ ]:
#verificando os 10 primeiros dados para ver
#iris = spark.read.schema(iris_schema).csv("/user/ronnan/data/iris/*.data").show(10)

In [ ]:
#lendo os arquivos com readStream
iris = spark.readStream.schema(iris_schema).csv("/user/ronnan/data/iris/*.data")

In [ ]:
#printar o schema para verificação
iris.printSchema()

In [ ]:
# saida do arquivo salvando como CSV, criando checkpoint com diretorio de metadados e path para escolher diretorio de salvamento
iris_saida = iris.writeStream.format("csv")\
    .option("checkpointLocation", "/user/ronnan/stream_iris/check")\
    .option("path","/user/ronnan/stream_iris/path")\
    .start()


In [ ]:
#ler o ultimo status do arquivo
iris_saida.lastProgress

In [ ]:
#ler os status do processo de saida
iris_saida.status

In [ ]:
#verificar no hdfs se os arquivos foram criados com diretorio de metadados
! hdfs dfs -ls /user/ronnan/stream_iris/check

In [ ]:
#verificar no hdfs se os arquivos foram criados no path
! hdfs dfs -ls /user/ronnan/stream_iris/path

In [ ]:
#ler dentro do diretorio para verificar se o arquivo contem as 150 linhas do inicio do exercicio
spark.read.csv ("/user/ronnan/stream_iris/path/part-00001-c85715cc-55f8-410f-896f-41dfc242d1a8-c000.csv").count()